In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
data_multi = np.loadtxt('..\\Data\\tictac_multi.txt')
#print(data_multi)
#print(np.shape(data_multi))
#print('\n')
names = ['k-NN Weighted']
max_a2 = 0
kk = 0
acc_fold = []
knnReg = KNeighborsRegressor(algorithm='kd_tree',n_neighbors=3, weights='distance')
kFold = KFold(n_splits=10, shuffle=True)
k_vals = np.arange(2,40,1)
for k in k_vals :
        X = data_multi[:,:9]
        Y = data_multi[:,9:]
        X = StandardScaler().fit_transform(X)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3)
        regs = [KNeighborsRegressor(k, weights = 'distance')]
        f = 1
        a2_val = 0
        for train_index, validation_index in kFold.split(X_train):
            x_train2, x_val = X_train[train_index], X_train[validation_index]
            y_train2, y_val = Y_train[train_index,:], Y_train[validation_index]

            for name, reg in zip(names,regs):
#                 print(f'Name {name}')
                reg.fit(x_train2, y_train2)
                y_val_predict = np.round(reg.predict(x_val))
                val_score = accuracy_score(y_val,y_val_predict)
#                 print(f'Validation Set Score: {val_score}')
                a2_val += val_score
            acc_fold += [a2_val/10]
            if a2_val/10 > max_a2:
                kk = k
                max_a2 = a2_val/10
            
            f += 1
print(f'The  k value chosen is {kk}, with an average score of {max_a2*100}%')
final_reg = KNeighborsRegressor(kk, weights = 'distance')
final_reg.fit(X_train, Y_train)
Y_pred = np.round(final_reg.predict(X_test))
score = accuracy_score(Y_pred, Y_test)
print(f'The score for the test set is {score*100}%') 



The  k value chosen is 29, with an average score of 77.55758198475895%
The score for the test set is 78.48423194303153%


In [4]:
def normalEq(x, y):
    b = np.ones((x.shape[0], 1))
    #print(b.shape)
    x_new = []
    for i in range(0,x.shape[1]):
        temp = np.reshape(x[:,i], (x.shape[0], 1))
        #print(temp.shape)
        if i == 0 :
            x_new = np.append(b, temp, axis = 1)
        else:
            x_new = np.append(x_new, temp, axis = 1)
    #print(x_new.shape)
    x_new_t = np.transpose(x_new)
    theta = np.linalg.inv(x_new_t.dot(x_new)).dot(x_new_t.dot(y.ravel()))
    #print(theta.shape)
    return theta
acc_scores = []
data_multi = np.loadtxt('..\\Data\\tictac_multi.txt')
X = data_multi[:,:9]
Y = data_multi[:,9:]
kFold = KFold(n_splits = 10, shuffle=True)
outputs = []
for train_index, test_index in kFold.split(X) :
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    params = []
    y_predicts = []
    for i in range(0,9):
        param = normalEq(x_train, y_train[:,i]).ravel()#643*9
        params.append(param)
        b = param[:-1]
        w = param[-1]
        y_predict = x_test.dot(w) + b#YT
        y_predicts.append(y_predict)#YTC
    y_predicts = np.array(y_predicts)
    for i in range(0, len(y_predicts[0])) :
        arr = y_predicts[:,i]
        mi = np.where(arr == np.amax(arr))
        y_predict[i, mi] = 1
    y_predict = np.argmax(np.floor(y_predict), axis = 1)
    acc = accuracy_score(np.argmax(y_test, axis=1), y_predict)
    acc_scores.append(acc)
print(acc_scores)
print("accuracy for linear regression :", np.mean(acc_scores))

[0.23323170731707318, 0.26564885496183205, 0.2381679389312977, 0.23053435114503817, 0.2534351145038168, 0.2198473282442748, 0.2381679389312977, 0.233587786259542, 0.22748091603053436, 0.24732824427480915]
accuracy for linear regression : 0.2387430180599516


In [10]:
from sklearn.neural_network import MLPRegressor

data_multi = np.loadtxt('..\\Data\\tictac_multi.txt')
X = data_multi[:,:9]
Y = data_multi[:,9:]
X_train, X_test, Y_train, Y_test = train_test_split(X,Y);
kFold = KFold(n_splits=10, shuffle=True)
mlpReg = MLPRegressor(activation='relu', hidden_layer_sizes=(100,80,80,80,80,80,100,), max_iter=100000, learning_rate_init=0.001)
acc_scores = []
for train_index, val_index in kFold.split(X_train):
    x_train, x_val = X_train[train_index], X_train[val_index]
    y_train, y_val = Y_train[train_index], Y_train[val_index]
    mlpReg.fit(x_train,y_train)
    y_val_predict = np.round(np.abs(mlpReg.predict(x_val)));
    accu = accuracy_score(np.argmax(y_val, axis=1),np.argmax(y_val_predict, axis=1))
    acc = mlpReg.score(x_val, y_val)
    acc_scores.append(acc)
print("mlp regressor accuracy: ", np.mean(acc_scores))
y_predict = mlpReg.predict(X_test);
fa = accuracy_score(np.argmax(Y_test, axis=1),np.argmax(y_predict, axis=1))
print("final accuracy: ", fa)

mlp regressor accuracy:  0.8012314148761253
final accuracy:  0.6892551892551892
